<a href="https://colab.research.google.com/github/samuelllopes/AnimaCursoExtensaoAnalisedeDadosPythonPandas-2023-1/blob/main/Aula04_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

combustiveis_df = pd.read_excel("ca-2021-02.xlsx")
display(combustiveis_df.head())

Inserção simples de dado

In [ ]:
combustiveis_df['Ativo'] = True

display(combustiveis_df.head())

Criando coluna Obs melhor cidade


In [ ]:
combustiveis_df['Obs'] = ["MELHOR CIDADE" if municipio == 'SAO PAULO' else None for municipio in combustiveis_df['Municipio']]
display(combustiveis_df.loc[combustiveis_df['Municipio'].isin(['SAO PAULO','INDAIATUBA', 'CAMPINAS', 'SALTO']), ['Municipio', 'Obs']])

Preenchimento de coluna com acima da média e abaixo

In [ ]:
import numpy as np

combustiveis_df['Valor de Venda - Status'] = np.where(combustiveis_df['Valor de Venda'] > 6.0, 'Acima da Média','Abaixo da Média')
display(combustiveis_df[['Valor de Venda - Status','Revenda','Valor de Venda']])

Qtd de postos por cidade

In [ ]:
word = "SAO PAULO"
count = combustiveis_df[combustiveis_df["Municipio"].str.contains(word)].shape[0]
display(count)


add base habitantes e trocando nome da coluna

In [ ]:
num_habitantes_df = pd.read_csv("ibge_num_habitantes_estimado.csv",sep=";")
num_habitantes_df.rename(columns={"Estado":"Estado - Sigla"}, inplace = True)
display(num_habitantes_df)

juntar duas bases

In [ ]:
colunas = ['Municipio', 'Estado - Sigla']
merge_df = combustiveis_df.merge(num_habitantes_df, how="inner", on=colunas)
display(merge_df)
print(merge_df.info())

Remover duplicadas 

In [ ]:
merge_df.drop_duplicates(inplace=True)
display(merge_df.head(100))

habitantes acima e abaixo da média

In [ ]:
merge_df['Status Habitantes'] = np.where(merge_df['NumHabitantes2021'] > 500000, 'Acima da População','Abaixo da População')
display(merge_df[['Status Habitantes','Estado - Sigla','Municipio', 'NumHabitantes2021']])

Agrupar e contar quantos postos tem na cidade

In [ ]:
postos_por_municipio_df = merge_df.groupby(by=['Estado - Sigla', 'Municipio', 'NumHabitantes2021']).count()
postos_por_municipio_df.reset_index(inplace=True)

postos_por_municipio_df.drop('CNPJ da Revenda', axis=1, inplace=True)
postos_por_municipio_df.rename(columns={"Revenda": "NumPostos"}, inplace=True)

postos_por_municipio_df['PostosPorHabitante'] = postos_por_municipio_df['NumPostos'] / postos_por_municipio_df['NumHabitantes2021']
display(postos_por_municipio_df.info())
display(postos_por_municipio_df)

Primeiro Gráfico

In [ ]:
import matplotlib.pyplot as plt

plt.hist(combustiveis_df['Valor de Venda'])
#Vamos colocar um título no gráfico
plt.title("Preço dos combustíveis - Nov/2021")
#Rótulo horizontal e vertical
plt.xlabel("Preço (em reais)")
plt.ylabel("Quantidade de Coletas")

#Traça a linha vermelha tracejada com o preço médio
plt.axvline(combustiveis_df['Valor de Venda'].mean(), color='red', linestyle='dashed', linewidth=5)


#"Plota" o gráfico
plt.show()

Consumo médio

In [ ]:
c_mean = combustiveis_df['Valor de Venda'].groupby(by=combustiveis_df['Produto']).mean()
display(c_mean)

Segundo Gráfico

In [ ]:
import seaborn as sns

#Vou definir a área do gráfico
plt.figure(figsize=(15,7))

#Plotar o gráfico
c_mean.plot(
    kind="barh",
    xlabel="Tipo de Combustível",
    ylabel="Preço reais/litro",
    title="Média de preços por combustível",
    color="red",
    alpha=0.3
)

#Grid
plt.grid()

#Remover as linhas superior e lateral direita do gráfico 
sns.despine()

#Exibe
plt.show()


Consumo médio

In [19]:
display(c_mean)

excel = "por_litro.xlsx"
c_mean.to_excel(excel, "Sumário")

Produto
DIESEL                5.375682
DIESEL S10            5.454901
ETANOL                5.456115
GASOLINA              6.708741
GASOLINA ADITIVADA    6.860422
GNV                   4.392935
Name: Valor de Venda, dtype: float64